# 데이터 불러오기 및 확인

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


기존 데이터 삽입

In [ ]:
import pandas as pd
import numpy as np

# aisles = pd.read_csv('/content/drive/MyDrive/AIB/cp1/aisles.csv')
# departments = pd.read_csv('/content/drive/MyDrive/AIB/cp1/departments.csv')
# order_products_prior = pd.read_csv('/content/drive/MyDrive/AIB/cp1/order_products__prior.csv')
order_products_train = pd.read_csv('/content/drive/MyDrive/AIB/cp1/order_products__train.csv')
orders = pd.read_csv('/content/drive/MyDrive/AIB/cp1/orders.csv')
products = pd.read_csv('/content/drive/MyDrive/AIB/cp1/products.csv')

cp1_feature_engineering(1)에서 만든 prior.csv와 user_prod_info.csv도 삽입

In [ ]:
# prior = pd.read_csv('/content/drive/MyDrive/AIB/cp1/prior.csv')
user_prod_info = pd.read_csv('/content/drive/MyDrive/AIB/cp1/user_prod_info.csv')

In [ ]:
orders.head()

order_id  user_id eval_set  order_number  order_dow  order_hour_of_day  \
0   2539329        1    prior             1          2                  8   
1   2398795        1    prior             2          3                  7   
2    473747        1    prior             3          3                 12   
3   2254736        1    prior             4          4                  7   
4    431534        1    prior             5          4                 15   

   days_since_prior_order  
0                     NaN  
1                    15.0  
2                    21.0  
3                    29.0  
4                    28.0

In [ ]:
prior.head()

order_id  user_id eval_set  order_number  order_dow  order_hour_of_day  \
0   2539329        1    prior             1          2                  8   
1   2539329        1    prior             1          2                  8   
2   2539329        1    prior             1          2                  8   
3   2539329        1    prior             1          2                  8   
4   2539329        1    prior             1          2                  8   

   days_since_prior_order  product_id  add_to_cart_order  reordered  
0                     0.0         196                  1          0  
1                     0.0       14084                  2          0  
2                     0.0       12427                  3          0  
3                     0.0       26088                  4          0  
4                     0.0       26405                  5          0

In [ ]:
user_prod_info.head()

user_id  product_id  up_order_num  up_reorder  up_avg_basketposition  \
0        1         196            10    0.900000               1.400000   
1        1       10258             9    0.888889               3.333333   
2        1       10326             1    0.000000               5.000000   
3        1       12427            10    0.900000               3.300000   
4        1       13032             3    0.666667               6.333333   

   u_order_num  u_avg_basket  u_most_dow  u_most_hod  u_reorder  u_avg_gap  \
0           10           5.9           4           7   0.694915  18.542373   
1           10           5.9           4           7   0.694915  18.542373   
2           10           5.9           4           7   0.694915  18.542373   
3           10           5.9           4           7   0.694915  18.542373   
4           10           5.9           4           7   0.694915  18.542373   

   u_product_num  p_order_num  p_reorder  p_avg_basketpositon  
0             59        35791   0.776480             3.721774  
1             59         1946   0.713772             4.277492  
2             59         5526   0.652009             4.191097  
3             59         6476   0.740735             4.760037  
4             59         3751   0.657158             5.622767

# 추가적인 feature engineering

orders에서 eval_set = train/test인 order_id를 추출(후에 order_products_train에서 타겟값으로 사용할 reordered 특성을 병합해야함)

In [ ]:
# train = orders.loc[(orders['eval_set'] == 'train'), ['user_id', 'eval_set', 'order_id']]
# train.head()

In [ ]:
train_test = orders.loc[((orders['eval_set'] == 'train')|(orders['eval_set'] == 'test')), ['user_id', 'eval_set', 'order_id']]
train_test.head()

user_id eval_set  order_id
10        1    train   1187899
25        2    train   1492625
38        3     test   2774568
44        4     test    329954
49        5    train   2196797

user_prod_info 데이터와 train_test 데이터를 user_id에 대해 병합하여 order_id 특성을 추가해줌

In [ ]:
# df2 = user_prod_info.merge(train, on = 'user_id', how = 'left')
# df2.head()

In [ ]:
df = user_prod_info.merge(train_test, on = 'user_id', how = 'left')
df.head()

user_id  product_id  up_order_num  up_reorder  up_avg_basketposition  \
0        1         196            10    0.900000               1.400000   
1        1       10258             9    0.888889               3.333333   
2        1       10326             1    0.000000               5.000000   
3        1       12427            10    0.900000               3.300000   
4        1       13032             3    0.666667               6.333333   

   u_order_num  u_avg_basket  u_most_dow  u_most_hod  u_reorder  u_avg_gap  \
0           10           5.9           4           7   0.694915  18.542373   
1           10           5.9           4           7   0.694915  18.542373   
2           10           5.9           4           7   0.694915  18.542373   
3           10           5.9           4           7   0.694915  18.542373   
4           10           5.9           4           7   0.694915  18.542373   

   u_product_num  p_order_num  p_reorder  p_avg_basketpositon eval_set  \
0             59        35791   0.776480             3.721774    train   
1             59         1946   0.713772             4.277492    train   
2             59         5526   0.652009             4.191097    train   
3             59         6476   0.740735             4.760037    train   
4             59         3751   0.657158             5.622767    train   

   order_id  
0   1187899  
1   1187899  
2   1187899  
3   1187899  
4   1187899

eval_set = test 인 데이터인 경우 reordered 특성을 넣어줄 수 없으므로(order_products_test 셋은 존재하지 않음)  
eval_set = train 인 데이터만 따로 가져옴

In [ ]:
df_train = df[df['eval_set'] == 'train']
df_train.head()

user_id  product_id  up_order_num  up_reorder  up_avg_basketposition  \
0        1         196            10    0.900000               1.400000   
1        1       10258             9    0.888889               3.333333   
2        1       10326             1    0.000000               5.000000   
3        1       12427            10    0.900000               3.300000   
4        1       13032             3    0.666667               6.333333   

   u_order_num  u_avg_basket  u_most_dow  u_most_hod  u_reorder  u_avg_gap  \
0           10           5.9           4           7   0.694915  18.542373   
1           10           5.9           4           7   0.694915  18.542373   
2           10           5.9           4           7   0.694915  18.542373   
3           10           5.9           4           7   0.694915  18.542373   
4           10           5.9           4           7   0.694915  18.542373   

   u_product_num  p_order_num  p_reorder  p_avg_basketpositon eval_set  \
0             59        35791   0.776480             3.721774    train   
1             59         1946   0.713772             4.277492    train   
2             59         5526   0.652009             4.191097    train   
3             59         6476   0.740735             4.760037    train   
4             59         3751   0.657158             5.622767    train   

   order_id  
0   1187899  
1   1187899  
2   1187899  
3   1187899  
4   1187899

order_product_train에서 order_id와 product_id에 대해서 reordered 데이터를 병합시킴

In [ ]:
df_train = df_train.merge(order_products_train[['product_id', 'order_id', 'reordered']], on = ['product_id', 'order_id'], how = 'left')
df_train.head()

user_id  product_id  up_order_num  up_reorder  up_avg_basketposition  \
0        1         196            10    0.900000               1.400000   
1        1       10258             9    0.888889               3.333333   
2        1       10326             1    0.000000               5.000000   
3        1       12427            10    0.900000               3.300000   
4        1       13032             3    0.666667               6.333333   

   u_order_num  u_avg_basket  u_most_dow  u_most_hod  u_reorder  u_avg_gap  \
0           10           5.9           4           7   0.694915  18.542373   
1           10           5.9           4           7   0.694915  18.542373   
2           10           5.9           4           7   0.694915  18.542373   
3           10           5.9           4           7   0.694915  18.542373   
4           10           5.9           4           7   0.694915  18.542373   

   u_product_num  p_order_num  p_reorder  p_avg_basketpositon eval_set  \
0             59        35791   0.776480             3.721774    train   
1             59         1946   0.713772             4.277492    train   
2             59         5526   0.652009             4.191097    train   
3             59         6476   0.740735             4.760037    train   
4             59         3751   0.657158             5.622767    train   

   order_id  reordered  
0   1187899        1.0  
1   1187899        1.0  
2   1187899        NaN  
3   1187899        NaN  
4   1187899        1.0

reordered에서 결측치가 있는데, 이는 product_id와 order_id에 대해 병합하는 과정에서  
해당 주문에서 그 제품은 주문하지 않았기에 발생한다  
즉, 해당 결측치는 0으로 대체할 수 있다

In [ ]:
df_train['reordered'] = df_train['reordered'].fillna(0)

reordered 컬럼을 병합하기 위해 사용한 eval_set, order_id 컬럼은 feature로 의미없다고 판단, 삭제  
(재주문 여부에 해당 주문번호가 영향이 있지는 않을 테니까)

In [ ]:
df_train.drop(['eval_set', 'order_id'], axis = 1, inplace = True)
df_train.head()

user_id  product_id  up_order_num  up_reorder  up_avg_basketposition  \
0        1         196            10    0.900000               1.400000   
1        1       10258             9    0.888889               3.333333   
2        1       10326             1    0.000000               5.000000   
3        1       12427            10    0.900000               3.300000   
4        1       13032             3    0.666667               6.333333   

   u_order_num  u_avg_basket  u_most_dow  u_most_hod  u_reorder  u_avg_gap  \
0           10           5.9           4           7   0.694915  18.542373   
1           10           5.9           4           7   0.694915  18.542373   
2           10           5.9           4           7   0.694915  18.542373   
3           10           5.9           4           7   0.694915  18.542373   
4           10           5.9           4           7   0.694915  18.542373   

   u_product_num  p_order_num  p_reorder  p_avg_basketpositon  reordered  
0             59        35791   0.776480             3.721774        1.0  
1             59         1946   0.713772             4.277492        1.0  
2             59         5526   0.652009             4.191097        0.0  
3             59         6476   0.740735             4.760037        0.0  
4             59         3751   0.657158             5.622767        1.0

최종 Feature engineering이 끝난 데이터셋을 data.csv라는 이름으로 저장

In [ ]:
df_train.to_csv('/content/drive/MyDrive/AIB/cp1/data.csv', index = False)

**최종 feature**  
- user_id : 개별 고객의 id
- product_id : 개별 상품의 id
- up_order_num : 고객이 특정 상품을 주문한 횟수
- up_reorder : 고객이 특정 상품을 재주문했던 확률(0~1사이의 값)
- up_avg_basketposition : 고객이 특정 상품을 장바구니에 담은 평균 순서
- u_order_num : 고객이 해당 쇼핑몰에서 주문한 횟수
- u_avg_basket : 고객이 주문마다 장바구니에 담은 상품의 평균 수
- u_most_dow : 고객이 가장 주문을 많이 한 요일
- u_most_hod : 고객이 가장 주문을 많이 한 시간대
- u_reorder : 고객이 재주문을 했던 확률(0~1사이의 값)
- u_avg_gap : 고객의 주문 평균 주기
- u_product_num : 고객이 주문을 한 상품의 총합
- p_order_num : 특정 상품이 전체고객들로부터 구매된 횟수
- p_reorder : 특정 상품이 전체 고객들로부터 재주문됐던 확률(0~1사이의 값)
- p_avg_basketposition : 특정 상품이 전체 고객들 장바구니에 담긴 평균 순서

**target**  
- reordered : 재주문 여부(0 or 1)